# Проектный практикум МФТИ - Детекция признаков депрессии по постам в социальных сетях

Проект разработан командой 8:

- Артамонов Олег
- Кулик Анастасия
- Семерня Элина

Для анализа текста используется предобученная модель deproberta-large-depression, классифицирующая текст на три категории:
- "Нет депрессии"
- "Умеренная депрессия"
- "Сильная депрессия"

Модель обрабатывает текст на английском языке. Для работы с русскими текстами применяется сервис машинного перевода Yandex Translate.



### Импорт библиотек и настройка переменных окружения
- Установите все необходимые пакеты
- Зарегистрируйтесь на Yandex Cloud для получения API-ключа и идентификатора папки, чтобы использовать машинный перевод с русского текста на английский

In [17]:
import os
import logging
import torch
from transformers import pipeline
from transformers import AutoModel, AutoTokenizer
from dataclasses import dataclass
import requests
from pprint import pprint

from dotenv import load_dotenv

In [ ]:
# Настройка логгера для вывода информации об ошибках
logging.basicConfig(
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
    level=logging.DEBUG
)
logger = logging.getLogger(__name__)

# Загрузка переменных окружения из .env файла
load_dotenv()


# Класс конфигурации для хранения настроек Yandex.Cloud
class Config:
    # API-ключ и идентификатор папки из Yandex.Cloud, полученные из переменных окружения
    YANDEX_API_KEY = os.getenv('YANDEX_API_KEY')  # Ключ API
    YANDEX_FOLDER_ID = os.getenv('YANDEX_FOLDER_ID')  # Идентификатор папки


# Проверка наличия необходимых настроек. Если хотя бы одна из них отсутствует, выдается ошибка.
if not Config.YANDEX_API_KEY or not Config.YANDEX_FOLDER_ID:
  raise ValueError('Необходима авторизация в Yandex.Cloud')

### Подключаем сервис машинного перевода Yandex Translate

In [ ]:
# Определяем структуру для хранения результата перевода.
@dataclass
class TranslationResult:
    translated_text: str
    detected_language: str
    success: bool

# Класс для работы с API Yandex Cloud Translation.
class YandexTranslationService:
    BASE_URL = 'https://translate.api.cloud.yandex.net/translate/v2'

    def __init__(self, api_key: str, folder_id: str):
        """
        Инициализация класса с использованием API-ключа и идентификатора папки.
        """
        self.api_key = api_key  # API-ключ для авторизации в Yandex Cloud
        self.folder_id = folder_id  # Идентификатор папки в Yandex Cloud
        # Заголовки для запросов к API
        self.headers = {
            'Authorization': f'Api-Key {api_key}',
            'Content-Type': 'application/json'
        }

    def translate_text(
            self,
            text: str,
            target_language: str = 'en',
            source_language: str = 'ru',
    ) -> TranslationResult:
        """
        Перевод текста с указанного исходного языка на целевой язык.
        Возвращает объект `TranslationResult` с результатами перевода.
        """
        try:
            # Подготовка данных для запроса
            payload = {
                'folder_id': self.folder_id,
                'texts': [text],
                'targetLanguageCode': target_language,
                'sourceLanguageCode': source_language,
            }

            # Отправка POST-запроса к API перевода
            response = requests.post(
                f'{self.BASE_URL}/translate',
                headers=self.headers,
                json=payload
            )

            # Проверяем, успешно ли завершился запрос
            response.raise_for_status()
            data = response.json()

            # Проверяем, есть ли переводы в ответе
            if not data.get('translations'):
                raise ValueError('Переводы не были получены при переводе из API')

            translation = data['translations'][0]

            # Возвращаем успешный результат
            return TranslationResult(
                translated_text=translation['text'],
                detected_language=translation.get('detectedLanguageCode', 'unknown'),
                success=True
            )

        except Exception as e:
            # Логируем ошибку, если что-то пошло не так
            logger.error(f'Ошибка при переводе: {str(e)}')
            return TranslationResult(
                translated_text='',
                detected_language='',
                success=False
            )

    def detect_language(self, text: str) -> str | None:
        """
        Определение языка исходного текста.
        Возвращает код языка или `None` в случае ошибки.
        """
        try:
            # Подготовка данных для запроса
            payload = {
                'folder_id': self.folder_id,
                'text': text
            }

            # Отправка POST-запроса к API определения языка
            response = requests.post(
                f'{self.BASE_URL}/detect',
                headers=self.headers,
                json=payload
            )

            # Проверяем, успешно ли завершился запрос
            response.raise_for_status()
            data = response.json()

            # Возвращаем код определенного языка
            return data.get('languageCode')

        except Exception as e:
            # Логируем ошибку, если что-то пошло не так
            logger.error(f'Ошибка при распознавании языка: {str(e)}')


# Создание экземпляра службы перевода с настройками из переменных окружения
translation_service = YandexTranslationService(
    api_key=Config.YANDEX_API_KEY,
    folder_id=Config.YANDEX_FOLDER_ID,
)


### Запуск модели deproberta-large-depression

In [12]:
model = pipeline(
    'text-classification',
    'rafalposwiata/deproberta-large-depression',
    device='cuda' if torch.cuda.is_available() else 'cpu',
)

In [16]:
# Пример использования модели
text = """
I'm very tired. Nothing pleases me in this life, I am tired of waking up every day to go to the office. I don't want to get out of bed.
"""
result = model(text)

print(result)

[{'label': 'moderate', 'score': 0.8198707699775696}]


### Функция перевода русского текста в английский и получения результата детекции модели

In [18]:
def get_depression_score(russian_text):
  translation_result: TranslationResult = translation_service.translate_text(
      text=source_text,
      target_language='en',
      source_language='ru',
  )

  english_text = translation_result.translated_text
  result = model(english_text)

  return result


In [ ]:
# Пример использования
source_text = """
Я очень устал, меня ничего не радует в этой жизни, я хочу это все закончить.
"""

score = get_depression_score(source_text)
print(score)

# Датасет текстов на русском языке

1. Dataset of depressive posts in Russian language collected from social media: https://www.sciencedirect.com/science/article/pii/S2352340920300895

dataset collected from social networks (VKontakte social network by using VKapi)

The collected data was classified by psychologists into two types: depressive and non-depressive. The dataset consists of 32 018 depressive posts and 32 021 non-depressive posts. Since the most common language that is spoken in CIS countries is Russian, the posts are written in Russian. The data can mostly be useful for researchers who explore tendencies to depression in CIS countries. The dataset is important for the research community, as it was not only collected from open sources, but also marked by our psychiatrists from the republican scientific and practical center of mental health. Since the dataset has very high validity, it can be used for further research in the field of mental health.


2. ВЫЯВЛЕНИЕ ПРИЗНАКОВ ДЕПРЕССИИ У АВТОРОВ ЭССЕ НА РУССКОМ ЯЗЫКЕ
Данная работа направлена на задачу выявления депрессии при по- мощи обработки и классификации 316 эссе. Коллекция из 93 эссе была предоставлена двумя коллективами психологов, которые попросили пациентов с клинически подтвержденной депрессией написать эссе на нейтральную тему. Остальные 223 эссе на аналогичную тему были написаны добровольцами, которые прошли стандартный опросник на выявление депрессии и не показали признаков наличия ментальных заболеваний. Исследование описывает различные психолингвисти- ческие и стандартные текстовые признаки, полученные при помощи инструментов обработки естественного языка и использованные для задачи классификации. Основанные на машинном обучении класси- фикационные модели продемонстрировали до 73% f1-меры в задаче обнаружения эссе, написанных людьми с депрессией.


In [ ]:
from google.colab import drive
import pandas as pd

# Монтируем Google Диск
drive.mount('/content/drive')

# Теперь вы можете открывать файлы из Google Диска
file_path = '/content/drive/My Drive/Colab Notebooks/Depressive data.xlsx'


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
dataset = pd.read_excel(file_path)

In [ ]:
dataset.head(5)

,text,label,age,text_cnt
0,"Когда-то я был добрым романтиком, который стре...",1,32.0,1699.0
1,Здраствуйте! Я каждый день просыпаюсь с мыслью...,1,28.0,769.0
2,У меня проблемы с девушкой. Каждую ссору я не ...,1,16.0,501.0
3,"Вся моя жизнь это один сплошной ад, в котором ...",1,32.0,236.0
4,Я хочу уснуть и не проснуться.каждый день одно...,1,14.0,2074.0


In [ ]:
dataset.tail(5)

,text,label,age,text_cnt
64034,Южная Корея будет внедрять свои технологии в К...,0,NaN,387.0
64035,В минувшие выходные в спортблоке КалмГУ прошли...,0,NaN,589.0
64036,ВТБ подвел итоги первого корпоративного акселе...,0,NaN,2431.0
64037,ВТБ запустит бесплатное пополнение карт других...,0,NaN,991.0
64038,"В Сызрани вновь пройдет ""Ярмарка вакансий"" для...",0,NaN,565.0


In [ ]:
print(dataset['text'][28])

print(dataset['label'][28])

Мне двадцать лет. Три года назад, сразу после школы, я уехала одна из Питера учиться в Израиль.
1


In [ ]:
dataset['text_cnt'] = dataset['text'].str.len()

In [ ]:
dataset['text_cnt'].sum()

69320725.0

In [ ]:
mini_dataset = dataset.sample(frac=0.012, random_state=1)  # random_state для воспроизводимости

In [ ]:
mini_dataset['text_cnt'].sum()

771129.0